In [3]:
from demoparser2 import DemoParser
import pandas as pd
import os
import sys

filename = "demos/0_0_0.dem"

filename_info = filename.split('/')[-1][:-4].split("_")
matchid = int(filename_info[0])
mapid = int(filename_info[1])
roundidoffset = int(filename_info[2])

parser = DemoParser(filename)

In [ ]:
player_kills_df = parser.parse_event(
    "player_death",
    player=["last_place_name", "team_name"],
    other=["total_rounds_played", "is_warmup_period"],
)


# get number of rounds played


num_rounds_played = (
    player_kills_df["total_rounds_played"].unique().tolist()[-1]
)  # we get last round + 1


# filter out team-kills and warmup


player_kills_df = player_kills_df[
    player_kills_df["attacker_team_name"] != player_kills_df["user_team_name"]
]


player_kills_df = player_kills_df[player_kills_df["is_warmup_period"] == False]

player_kills_df = (
    player_kills_df.groupby(["total_rounds_played", "attacker_name"])
    .size()
    .to_frame(name="kills")
    .reset_index()
)


player_list = parser.parse_player_info()["name"].tolist()

# rename columns for clarity
player_kills_df = player_kills_df.rename(
    columns={
        "total_rounds_played": "round_id",
        "attacker_name": "player_name",
        "kills": "player_kills",
    }
)

# output to txt file


output_filename = filename.replace(".dem", ".csv")


output_filename = output_filename.replace("demos", "rinaldo_csv_toilet")


player_kills_df.to_csv(output_filename, index=False)

In [ ]:
"""
ChatGPT was used to assist in understanding dataframe manipulation techniques in Pandas.
Much of the actual code following was written by Sean Bernardo.
"""


part = "head"
rounds = 17

player_shots_df = parser.parse_event("player_hurt")

# This I robbed get_round_start_times which is rinaldo's code but instead i change it to tick so no need for more conversions
# because of how bins work, we add the last tick of the demo to accommodate the last round
round_start_times = list(
    parser.parse_ticks(
        ["game_time"], ticks=parser.parse_event("round_freeze_end")["tick"]
    )
    .drop_duplicates(subset=["tick"])["tick"]
    .values
) + [parser.parse_ticks(["tick"]).iloc[-1]["tick"]]


# We create a surrogate column for round_id based on round start times
player_shots_df["round_id"] = pd.cut(
    player_shots_df["tick"], bins=round_start_times, labels=False, right=True
)

"""
Inspecting the demo we get the following hitgroups: ['generic', 'head', 'chest', 'left_arm', 'stomach', 'right_arm', 'left_leg', 'right_leg', 'neck']

Groupings are determined by testing damage done to each hitgroup.

The following console commands were used to determine damage applied and aim specific hitbox groups:
    player_debug_print_damage true
    cl_ent_hitbox *

Each hitbox group was tested 5 times and the average damage was recorded rounded to the nearest integer.

Head:       205

Upper Body
- Neck:      48
- Chest:     48
- Right Arm: 48
- Left Arm:  48

- Stomach:   60

Legs
- Right Leg: 38
- Left Leg:  38

We drop generic as this usually covers fall damage and utils
"""


hitgroups = {
    "head": ["head"],
    "upperbody": ["neck", "chest", "right_arm", "left_arm"],
    "stomach": ["stomach"],
    "legs": ["right_leg", "left_leg"],
}

# group by round_id and attacker_name

if part not in hitgroups:
    pass

player_shots_df = player_shots_df[player_shots_df["hitgroup"].isin(hitgroups[part])]

player_shots_df = player_shots_df.groupby(["round_id", "attacker_name"]).size().reset_index(
    name=f"player_{part}shots"
)

player_shots_df.to_csv("rinaldo_csv_toilet/wtf.csv", index=False)

In [ ]:
round_start_times = list(
    parser.parse_ticks(
        ["game_time"], ticks=parser.parse_event("round_freeze_end")["tick"]
    )
    .drop_duplicates(subset=["tick"])["tick"]
    .values
) + [parser.parse_ticks(["tick"]).iloc[-1]["tick"]]

bomb_plants_df = parser.parse_event("bomb_planted", player=["player_name", "team_name"])

# We create a surrogate column for round_id based on round start times
bomb_plants_df["round_id"] = pd.cut(
    bomb_plants_df["tick"], bins=round_start_times, labels=False, right=True
)

bomb_plants_df = bomb_plants_df[["user_name", "round_id"]]

bomb_plants_df

,user_name,round_id
0,Wicadia,0
1,Wicadia,1
2,woxic,2
3,woxic,5
4,woxic,6
5,MAJ3R,7
6,woxic,9
7,Jeorge,15
